# Notebook Objective and Setup

BGG05 is the building of a content-based item filter. Using category weights, I use my domain expertise to tune an item similarity matrix for all game IDs in the games file.

This content-based filter could be used as-is to find similar games to a user's catalog and predict ratings.

## Notebook Preparation

### Package Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import regex as re
import time
import os
import gc
import copy
import json

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from missingpy import MissForest
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfTransformer

import umap
import umap.plot
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster
%matplotlib inline

### Notebook Functions

##### Processing Functions

In [2]:
def processing_pipeline(weight_groups, df):
    '''
    !!!Hard-coded processor!!!
    Takes in weight tuples and a dataframe
    Scales specific dataframe columns to tuples
    
    Inputs:
    weight_groups: list of weight tuples (x, y)
    df: df to be scaled
    
    Returns:
    Processed Dataframe'''

   
    # continuous pipeline
    family_encoder = Pipeline([
        ('encoder', OneHotEncoder()),
        ('scaler', MinMaxScaler(feature_range=weight_groups[6])),
         ])
    
    # Whole pipeline with continuous then categorical transformers
    total_pipeline = ColumnTransformer([
        ('games_weight_weight', MinMaxScaler(feature_range=weight_groups[0]), ['GameWeight']),
        ('avgrating', MinMaxScaler(feature_range=weight_groups[1]), ['AvgRating']),
        ('bayes_weight', MinMaxScaler(feature_range=weight_groups[2]), ['BayesAvgRating']),  
        ('players_weight', MinMaxScaler(feature_range=weight_groups[3]), ['BestPlayers']),
        ('playtime_weight', MinMaxScaler(feature_range=weight_groups[4]), ['Playtime']),
        ('remainder_weight', MinMaxScaler(feature_range=weight_groups[5]), [ 'Cat:War',
                       'Cat:CGS','Cat:Abstract','Cat:Party','Cat:Childrens']),
                            ]) 
    
    # Fit and tranform the pipeline on x_train, then transform x_test
    processed = total_pipeline.fit_transform(df)
    
    return processed

In [3]:
def scale_dataset(dataset, weights, tfidf=False):
    '''
    Set each item either according to Term Frequency or TF-IDF
    Then scale dataset according to provided weights
    
    tfidf flag:
    Use TRUE when RARE entries are more important than FREQUENT entries
    Use FALSE when COMMON entries are more important than RARE entries
    
    Inputs:
    dataset: dataset to scale
    weights: tuple (x, y) to weight dataset
    tfidf: set flag to True for TF-IDF
    
    Outputs:
    Scaled data
    '''
    
    # drop BGG Id
    try: 
        dataset_pared = dataset.drop('BGGId', axis=1)
    except: 
        dataset_pared = dataset
    
    # get list of titles to reapply to DF after transformation
    titles = list(dataset_pared.columns)

    # set up weighted scaler
    scaler = MinMaxScaler(feature_range = weights)
    
    # get total number of entries
    total_entries = sum(dataset.sum())
    
    if tfidf:
        
        #instantiate tfidf transformer
        tfidf = TfidfTransformer()
    
        #convert matrix to tfidf 
        tfidf_dataset = pd.DataFrame(tfidf.fit_transform(dataset_pared).toarray(), columns=titles)
        
        # run scaler on transpose (scale by row not column)
        transpose_scaled = scaler.fit_transform(tfidf_dataset.T)
        
    else:
        # for each column,
        for item in list(dataset_pared.columns):
            # set item as its term frequency
            dataset_pared.loc[dataset_pared[item]>0, item] = dataset_pared[item].sum()/total_entries
    
        # run scaler on transpose (scale by row not column)
        transpose_scaled = scaler.fit_transform(dataset_pared.T)
    
    # rebuild dataframe
    scaled_dataset = pd.DataFrame(transpose_scaled.T, columns=titles)
    
    return scaled_dataset


# Content Based Filtering

## Set Weights

These are the scales for each of these categories. All entries in the category will be scaled to this tuple range by the MinMaxScaler.

In [4]:
### DO NOT TOUCH THESE ARE THE PRODUCTION WEIGHTS!!!!!

games_weight_weight = (-1.25, 1.25) # game weight. Is a range, so (-, )
rating_weight = (-.3, .3)
bayes_weight = (-.3, .3) # game weighted rating. Is a range, so (-, )
players_weight = (0, .5) # best players. Is a problematic range due to outliers
playtime_weight = (0, 1.5) # playtime. Is a range so (-, ). Has high outliers
designers_weight = (0, 0.4) # designers, binary
mechanics_weight = (0, .75) # mechanics, binary
subcategories_weight = (0, 0.75) # other mechanics like card game, print&play. binary
family_weights = (0, 0.5) # game families like pandemic, century. binary
categories_weight = (0, 0.75) # the five large overarching categories, binary
themes_weight = (-.1, 0.1) # themes like space, western. binary

In [56]:
# these are the weights that can be played with

games_weight_weight = (-1.5, 1.5) # game weight. Is a range, so (-, )
rating_weight = (-.25, .25)
bayes_weight = (-.5, .5) # game weighted rating. Is a range, so (-, )
players_weight = (0, .5) # best players. Is a problematic range due to outliers
playtime_weight = (0, 2) # playtime. Is a range so (-, ). Has high outliers
designers_weight = (0, 0.5) # designers, binary
mechanics_weight = (0, .75) # mechanics, binary
subcategories_weight = (0, .75) # other mechanics like card game, print&play. binary
family_weights = (0, 0.75) # game families like pandemic, century. binary
categories_weight = (0, 1) # the five large overarching categories, binary
themes_weight = (-.01, 0.01) # themes like space, western. binary

## Load and Prep Data

In [5]:
# Load games
games = pd.read_pickle('data_cleaned/games.pkl')

# determine playtime for each game according to community
games['Playtime'] = 0
games['Playtime'] = games.apply(lambda x: np.mean(x['ComMinPlaytime'] + x['ComMaxPlaytime']), axis=1)

# set upper cap on playtime
over_6_hours = list(games.loc[games['Playtime']>360].index)
games.loc[over_6_hours, 'Playtime']=360

# load other files to use
mechanics = pd.read_pickle('data_cleaned/mechanics.pkl')
designers = pd.read_pickle('data_cleaned/designers_reduced.pkl')
subcategories = pd.read_pickle('data_cleaned/subcategories.pkl')
themes = pd.read_pickle('data_cleaned/themes.pkl')

games.head()

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumAwards,NumImplementations,NumFans,NumPageViews,RulesPosts,TotalPosts,IsExpansion,IsReimplementation,Family,Kickstarted,ImagePath,Rank:boardgame,Rank:thematic,Rank:strategygames,Rank:wargames,Rank:familygames,Rank:cgs,Rank:abstracts,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens,Playtime
0,174430,Gloomhaven,gloomhaven game euroinspired tactical combat...,2017,3.8726,8.76029,8.52385,1.63358,1,4,12.784946,4.152542,3,"[1, 2, 3, 4]",74975,1375,17276,2009,120,60,120,14,46228,8308,6,12,30,0,7718,10816029,6326,14752,0,0,Gloomhaven,1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,1,1,1,21369,21369,21369,21369,21369,21369,1,1,0,0,0,0,0,0,180.0
1,161936,Pandemic Legacy: Season 1,pandemic legacy cooperative campaign game over...,2015,2.8331,8.60116,8.45000,1.56368,2,4,11.354037,4.077778,4,"[2, 3, 4]",68974,840,11544,1210,60,60,60,13,44043,6682,10,0,30,2,2952,3450336,1229,3086,0,1,Pandemic,0,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,2,2,3,21369,21369,21369,21369,21369,21369,1,1,0,0,0,0,0,0,120.0
2,224517,Brass: Birmingham,brass birmingham economic strategy game sequel...,2018,3.9038,8.66907,8.41066,1.24331,2,4,13.226190,1.035714,3,"[2, 3, 4]",35448,1528,11277,1143,120,60,120,14,23776,3610,6,0,18,1,1902,1974872,387,1028,0,1,Brass,1,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,3,21369,2,21369,21369,21369,21369,21369,21369,0,1,0,0,0,0,0,0,180.0
3,167791,Terraforming Mars,s mankind begin terraform planet mar giant cor...,2016,3.2429,8.42155,8.27751,1.38628,1,5,11.891156,3.380952,3,"[1, 2, 3, 4]",97330,2061,18842,2944,120,120,120,12,71474,10452,14,22,39,2,6372,6285054,1155,4313,0,0,Terraforming Mars,0,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZ...,4,21369,6,21369,21369,21369,21369,21369,21369,0,1,0,0,0,0,0,0,240.0
4,291457,Gloomhaven: Jaws of the Lion,gloomhaven jaw lion standalone game take place...,2020,3.5649,8.72198,8.25902,1.42169,1,4,12.051948,4.000000,2,"[1, 2, 3, 4]",33444,477,6250,485,120,30,120,14,13939,2149,6,0,11,0,1649,1471587,1073,2062,0,0,Gloomhaven,0,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq...,5,3,5,21369,21369,21369,21369,21369,21369,1,1,0,0,0,0,0,0,150.0


### Weight Scale Datasets

##### TF-IDF Mechanics

In [6]:
# get floor of mechanics presence in catalog (.05% of games)
mechanics_floor = round(mechanics.shape[0]*.005)

# make a list of mechanics more than the floor
sums = pd.DataFrame(mechanics.sum()>=mechanics_floor)

# get indices for the mechanics keeping
keep_mechanics = sums.loc[sums[0]==True].index

refined_mechanics = mechanics[keep_mechanics]

# scaled mechanics
scaled_mechanics = scale_dataset(refined_mechanics, mechanics_weight, tfidf=True)

# make new column for games without any mechanics information
no_mechanics_index = list(scaled_mechanics.loc[scaled_mechanics.sum(axis=1)==0].index)
scaled_mechanics['No Mechanics'] = 0
scaled_mechanics.loc[no_mechanics_index, 'No Mechanics'] = (mechanics_weight[1])

##### TF-Scale Only Datasets

In [7]:
# scale themes
scaler = MinMaxScaler(feature_range=themes_weight)
scaled_themes = scaler.fit_transform(themes)

# get list of titles to reapply to DF after transformation
titles = list(themes.columns)  

scaled_themes = pd.DataFrame(scaled_themes, columns=titles)
scaled_themes.drop('BGGId', axis=1, inplace=True)

# scaled designers
scaled_designers = scale_dataset(designers, designers_weight)

# scaled game families
game_families = pd.get_dummies(games['Family'])
scaled_families = scale_dataset(game_families, family_weights)

#scaled subcategories
scaled_subcategories = scale_dataset(subcategories, subcategories_weight)

### Master CBF Frame

In [8]:
# include these columns for comparison
games_included_columns = ['GameWeight', 'AvgRating', 'BayesAvgRating', 'BestPlayers', 'Playtime', 'Cat:War', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens'] 

# make smaller df of the included columns
scaled_games = games[games_included_columns]

# get list of game names
game_names = list(games['Name'])
# get list of game ids
game_ids = list(games['BGGId'])

# create game lookup table
game_lookup = {}
for key, value in zip(game_ids, game_names):
    game_lookup[key] = value

# instantiate MissForest imputer and fill all nans in scaled_games
imputer = MissForest()
scaled_games = pd.DataFrame(imputer.fit_transform(scaled_games), columns=games_included_columns)

In [9]:
# set up weight groups for hard coded pipeline
weight_groups = [games_weight_weight, rating_weight, bayes_weight, players_weight, playtime_weight, categories_weight, family_weights]

# process scaled_games with pipeline
scaled_games = pd.DataFrame(processing_pipeline(weight_groups, scaled_games), columns=games_included_columns)

# put together master dataframe with other already processed dataframes
master_games = pd.concat((scaled_games, scaled_themes, scaled_mechanics, scaled_families, scaled_designers, scaled_subcategories), axis=1) #, scaled_themes

# make list of games and ids (is this used anywhere?)
game_and_id = list(zip(game_names, game_ids))

# put game id on master_games DF
master_games['BGGId'] = game_ids

# set index to id
master_games.set_index('BGGId', inplace=True)

master_games.head()

GameWeight  AvgRating  BayesAvgRating  BestPlayers  Playtime  Cat:War  \
BGGId                                                                           
174430     0.68630   0.240157        0.300000     0.100000     0.750      0.0   
161936     0.16655   0.229021        0.291052     0.133333     0.500      0.0   
224517     0.70190   0.233774        0.286286     0.100000     0.750      0.0   
167791     0.37145   0.216451        0.270153     0.100000     1.000      0.0   
291457     0.53245   0.237476        0.267913     0.066667     0.625      0.0   

        Cat:CGS  Cat:Abstract  Cat:Party  Cat:Childrens  Adventure  Fantasy  \
BGGId                                                                         
174430      0.0           0.0        0.0            0.0        0.1      0.1   
161936      0.0           0.0        0.0            0.0       -0.1     -0.1   
224517      0.0           0.0        0.0            0.0       -0.1     -0.1   
167791      0.0           0.0        0.0            0.0       -0.1     -0.1   
291457      0.0           0.0        0.0            0.0        0.1      0.1   

        Fighting  Environmental  Medical  Economic  Industry / Manufacturing  \
BGGId                                                                          
174430       0.1           -0.1     -0.1      -0.1                      -0.1   
161936      -0.1            0.1      0.1      -0.1                      -0.1   
224517      -0.1           -0.1     -0.1       0.1                       0.1   
167791      -0.1            0.1     -0.1       0.1                       0.1   
291457       0.1           -0.1     -0.1      -0.1                      -0.1   

        Transportation  Science Fiction  Space Exploration  Civilization  \
BGGId                                                                      
174430            -0.1             -0.1               -0.1          -0.1   
161936            -0.1             -0.1               -0.1          -0.1   
224517             0.1             -0.1               -0.1          -0.1   
167791            -0.1              0.1                0.1          -0.1   
291457            -0.1             -0.1               -0.1          -0.1   

        Civil War  Movies / TV / Radio theme  Novel-based  Age of Reason  \
BGGId                                                                      
174430       -0.1                       -0.1         -0.1           -0.1   
161936       -0.1                       -0.1         -0.1           -0.1   
224517       -0.1                       -0.1         -0.1           -0.1   
167791       -0.1                       -0.1         -0.1           -0.1   
291457       -0.1                       -0.1         -0.1           -0.1   

        Mythology  Renaissance  American West  Animals  Modern Warfare  \
BGGId                                                                    
174430       -0.1         -0.1           -0.1     -0.1            -0.1   
161936       -0.1         -0.1           -0.1     -0.1            -0.1   
224517       -0.1         -0.1           -0.1     -0.1            -0.1   
167791       -0.1         -0.1           -0.1     -0.1            -0.1   
291457       -0.1         -0.1           -0.1     -0.1            -0.1   

        Medieval  Ancient  Nautical  Post-Napoleonic  Horror  Farming  \
BGGId                                                                   
174430      -0.1     -0.1      -0.1             -0.1    -0.1     -0.1   
161936      -0.1     -0.1      -0.1             -0.1    -0.1     -0.1   
224517      -0.1     -0.1      -0.1             -0.1    -0.1     -0.1   
167791      -0.1     -0.1      -0.1             -0.1    -0.1     -0.1   
291457      -0.1     -0.1      -0.1             -0.1    -0.1     -0.1   

        Religious  Travel  Murder/Mystery  Pirates  Comic Book / Strip  \
BGGId                                                                    
174430       -0.1    -0.1            -0.1     -0.1                -0.1   
161936

In [10]:
# clean up

del mechanics
del designers
del subcategories
del themes
del scaled_mechanics
del scaled_families
del scaled_designers
del scaled_games
del scaled_subcategories
del scaled_themes

gc.collect()

1550

## Item Similarity via Cosine Distance

In [11]:
# get cosine similarities!
cosine_sims = cosine_similarity(master_games)

# do similarities by game id
sims_byid = pd.DataFrame(cosine_sims, columns=game_ids)
sims_byid['Game_Id'] = game_ids
sims_byid.set_index('Game_Id', inplace=True, drop=True)
#sims_byid

In [12]:
sims_byid = sims_byid.round(6)

for column in sims_byid.columns:
    lower = sims_byid[column].min()
    sims_byid[column].replace(1.,lower, inplace=True)
    if sims_byid[column].max() > .96:
        continue
    else:
        scaler = MinMaxScaler(feature_range=(lower, .96))
        scaled_values = scaler.fit_transform(np.array(sims_byid[column]).reshape(-1,1))
        sims_byid[column] = scaled_values

In [13]:
sims_byname = sims_byid.copy()
sims_byname.set_axis(game_names, axis=1, inplace=True)
sims_byname.set_axis(game_names, axis=0, inplace=True)

In [14]:
# save to pickles, we really only need the id one
#sims_byid.to_pickle('data_cleaned/game_cosine_similarity_byid.pkl')
#sims_byname.to_pickle('data_cleaned/game_cosine_similarity_byname.pkl')

### CHECK GAME HERE

In [15]:
# save to pickles, we really only need the id one
#sims_by_id = pd.read_pickle('data_cleaned/game_cosine_similarity_byid.pkl')
#sims_byname = pd.read_pickle('data_cleaned/game_cosine_similarity_byname.pkl')

This is why we made the name one

In [16]:
# test my specific game set here

test_dict = {'Dominion':list(sims_byname['Dominion'].sort_values(ascending=False)[:15].index), 'D_Sim':list(sims_byname['Dominion'].sort_values(ascending=False)[:15]),
            'Gloomhaven':list(sims_byname['Gloomhaven'].sort_values(ascending=False)[:15].index), 'G_Sim':list(sims_byname['Gloomhaven'].sort_values(ascending=False)[:15]),
            'Pandemic':list(sims_byname['Pandemic'].sort_values(ascending=False)[:15].index), 'Pa_Sim':list(sims_byname['Pandemic'].sort_values(ascending=False)[:15]),
            'Splendor':list(sims_byname['Splendor'].sort_values(ascending=False)[:15].index), 'Sp_Sim':list(sims_byname['Splendor'].sort_values(ascending=False)[:15]),
            'Viticulture Essential Edition':list(sims_byname['Viticulture Essential Edition'].sort_values(ascending=False)[:15].index), 'V_Sim':list(sims_byname['Viticulture Essential Edition'].sort_values(ascending=False)[:15]),
            'Agricola':list(sims_byname['Agricola'].sort_values(ascending=False)[:15].index), 'Ag_Sim':list(sims_byname['Agricola'].sort_values(ascending=False)[:15]),
             'Space Base':list(sims_byname['Space Base'].sort_values(ascending=False)[:15].index), 'Spa_Sim':list(sims_byname['Space Base'].sort_values(ascending=False)[:15]),
             'Terraforming Mars':list(sims_byname['Terraforming Mars'].sort_values(ascending=False)[:15].index), 'Te_Sim':list(sims_byname['Terraforming Mars'].sort_values(ascending=False)[:15]),
            'Puerto Rico':list(sims_byname['Puerto Rico'].sort_values(ascending=False)[:15].index), 'Pu_Sim':list(sims_byname['Puerto Rico'].sort_values(ascending=False)[:15]),
            'Chess':list(sims_byname['Chess'].sort_values(ascending=False)[:15].index), 'Ch_Sim':list(sims_byname['Chess'].sort_values(ascending=False)[:15]),
            'Backgammon':list(sims_byname['Backgammon'].sort_values(ascending=False)[:15].index), 'B_Sim':list(sims_byname['Backgammon'].sort_values(ascending=False)[:15]),
            'Sagrada':list(sims_byname['Sagrada'].sort_values(ascending=False)[:15].index), 'Sa_Sim':list(sims_byname['Sagrada'].sort_values(ascending=False)[:15]),
            'Azul':list(sims_byname['Azul'].sort_values(ascending=False)[:15].index), 'Az_Sim':list(sims_byname['Azul'].sort_values(ascending=False)[:15]),
            'Codenames':list(sims_byname['Codenames'].sort_values(ascending=False)[:15].index), 'Co_Sim':list(sims_byname['Codenames'].sort_values(ascending=False)[:15]),
            'Secret Hitler':list(sims_byname['Secret Hitler'].sort_values(ascending=False)[:15].index), 'Se_Sim':list(sims_byname['Secret Hitler'].sort_values(ascending=False)[:15]),
            'Monopoly':list(sims_byname['Monopoly'].sort_values(ascending=False)[:15].index), 'M_Sim':list(sims_byname['Monopoly'].sort_values(ascending=False)[:15]), 
            'Lords of Waterdeep':list(sims_byname['Lords of Waterdeep'].sort_values(ascending=False)[:15].index), 'L_Sim':list(sims_byname['Lords of Waterdeep'].sort_values(ascending=False)[:15]),
            'Stone Age':list(sims_byname['Stone Age'].sort_values(ascending=False)[:15].index), 'St_Sim':list(sims_byname['Stone Age'].sort_values(ascending=False)[:15]),
            'Century: Spice Road':list(sims_byname['Century: Spice Road'].sort_values(ascending=False)[:15].index), 'Ce_Sim':list(sims_byname['Century: Spice Road'].sort_values(ascending=False)[:15]),
            'Scrabble':list(sims_byname['Scrabble'].sort_values(ascending=False)[:15].index), 'Sc_Sim':list(sims_byname['Scrabble'].sort_values(ascending=False)[:15]),
            '18MS: The Railroads Come to Mississippi':list(sims_byname['18MS: The Railroads Come to Mississippi'].sort_values(ascending=False)[:15].index), '18xx':list(sims_byname['18MS: The Railroads Come to Mississippi'].sort_values(ascending=False)[:15]),
             'Roads to Gettysburg II: Lee Strikes North':list(sims_byname['Roads to Gettysburg II: Lee Strikes North'].sort_values(ascending=False)[:15].index), 'War3':list(sims_byname['Roads to Gettysburg II: Lee Strikes North'].sort_values(ascending=False)[:15]),
             'Power Grid':list(sims_byname['Power Grid'].sort_values(ascending=False)[:15].index), 'P_Grid':list(sims_byname['Power Grid'].sort_values(ascending=False)[:15]),
            }

pd.DataFrame(test_dict)

,Dominion,D_Sim,Gloomhaven,G_Sim,Pandemic,Pa_Sim,Splendor,Sp_Sim,Viticulture Essential Edition,V_Sim,Agricola,Ag_Sim,Space Base,Spa_Sim,Terraforming Mars,Te_Sim,Puerto Rico,Pu_Sim,Chess,Ch_Sim,Backgammon,B_Sim,Sagrada,Sa_Sim,Azul,Az_Sim,Codenames,Co_Sim,Secret Hitler,Se_Sim,Monopoly,M_Sim,Lords of Waterdeep,L_Sim,Stone Age,St_Sim,Century: Spice Road,Ce_Sim,Scrabble,Sc_Sim,18MS: The Railroads Come to Mississippi,18xx,Roads to Gettysburg II: Lee Strikes North,War3,Power Grid,P_Grid
0,Dominion: Intrigue,0.960000,Gloomhaven: Jaws of the Lion,0.960000,Pandemic Legacy: Season 1,0.960000,Splendor: Marvel,0.960000,Viticulture,0.960000,Agricola (Revised Edition),0.964904,Seasons,0.960000,Gaia Project,0.960000,Glory to Rome,0.960000,Shogi,0.960000,Senet,0.960000,Cóatl,0.960000,Azul: Summer Pavilion,0.966526,Codenames: Pictures,0.999173,Wherewolf,0.960000,Monopoly: My Marvel Heroes,0.960000,Steam Time,0.960000,Far Space Foundry,0.960000,Century: Golem Edition,0.986664,Scarabeo,0.960000,18Dixie with 18MS and 18GA,0.997475,Stonewall Jackson's Way,0.960000,Power Grid Deluxe: Europe/North America,0.960000
1,Dominion (Second Edition),0.896719,Middara: Unintentional Malum – Act 1,0.763568,Pandemic: Reign of Cthulhu,0.942410,Chai,0.861806,Corrosion,0.956055,Paladins of the West Kingdom,0.767458,Santa Monica,0.932482,Corduba 27 a.C.,0.948425,Russian Railroads,0.935066,Chess960,0.918615,Obsession,0.935548,Mosaix,0.836849,Azul: Stained Glass of Sintra,0.860713,Codenames: Deep Undercover,0.986428,Ultimate Werewolf: Ultimate Edition,0.923733,Monopoly: Game of Thrones Collector's Edition,0.927407,Century: Golem Edition – An Endless World,0.902166,Merchants of the Dark Road,0.911414,Kashgar: Merchants of the Silk Road,0.806862,Equate,0.946592,18MEX,0.953252,Stonewall Jackson's Way II: Battles of Bull Run,0.937321,Mégawatts,0.949121
2,Pentaurus: Duel,0.869408,Etherfields,0.759791,Pandemic: Iberia,0.937659,Gizmos,0.838891,Raiders of Scythia,0.918566,Anachrony,0.752963,Elysium,0.927935,Terra Mystica,0.904095,The Princes of Florence,0.894403,Go,0.916050,Martian Backgammon,0.910515,Orion,0.825978,Azul: Queen's Garden,0.837840,Codenames: Disney – Family Edition,0.945332,The Chameleon,0.921623,Superpoly,0.926570,Alubari: A Nice Cup of Tea,0.871812,Ragusa,0.876402,Gelato Mio,0.802535,Last Word,0.932112,1879: US Pacific Northwest,0.948607,Stonewall in the Valley,0.931063,Funkenschlag: EnBW,0.947993
3,Het Koninkrijk Dominion,0.866351,Bloodborne: The Board Game,0.733434,World of Warcraft: Wrath of the Lich King,0.933615,Quests of Valeria,0.808084,Ore: The Mining Game,0.870498,Caverna: The Cave Farmers,0.741509,London,0.924773,Suburbia,0.899076,The Patrons of Venice,0.884892,International Checkers,0.904426,Diary of a Wimpy Kid: Cheese Touch,0.908980,Realm of Sand,0.815403,Tantrix,0.775229,Codenames: Marvel,0.828383,Shadow Hunters,0.904544,Monopoly: Friends The TV Series,0.923535,Raiders of the North Sea,0.871702,Yokohama,0.856387,Vidrado,0.788880,Scrabble: Harry Potter Edition,0.917414,1876: Trinidad,0.944889,Here Come the Rebels,0.930289,Funkenschlag,0.942985
4,Outpost 18,0.863398,EXO: Mankind Reborn,0.719642,Pandemic Legacy: Season 0,0.871044,Port Royal,0.801127,Alubari: A Nice Cup of Tea,0.869735,Fields of Arle,0.740697,Oceans,0.917863,Scythe,0.893602,Corduba 27 a.C.,0.880691,Dipole,0.889172,Gemlok,0.901984,Noctiluca,0.812746,Seikatsu,0.772912,Codenames: Duet,0.826967,Shinobi: El Juego del Asesino,0.900020,Monopoly: Canadian Edition,0.922225,Finca,0.871217,Shipyard,0.855177,Space Battle Lunchtime Card Game,0.775572,Wordigo,0.892206,18TN,0.944207,The Siege of Jerusalem (Third Edition),0.929866,Homesteaders,0.802107
5,Eternal: Chronicles of the Throne,0.851810,The Lord of the Rings: Journeys in Middle-Earth,0.716329,Pandemic: Fall of Rome,0.816024,Amul,0.767984,Hallertau,0.865193,A Feast for Odin,0.725496,Unfair,0.915813,Newton,0.870549,Exodus Fleet,0.877353,Hnefatafl,0.881907,Shake Up,0.898807,Nova Luna,0.805241,Tantrix Gobble,0.762975,Medium,0.811088,The Last 

In [17]:
# test specific games here
game = 'Dinosaur Island'

results = pd.DataFrame(data={'Similarity': sims_byname[game].sort_values(ascending=False)[0:21]})
results

,Similarity
Lowlands,0.960000
Neuland,0.884339
Mi Tierra: New Era,0.845580
Coal Baron,0.836999
Dinosaur World,0.831561
Signorie,0.824254
Honga,0.822005
Way Out West,0.816494
Tales of the Northlands: The Sagas of Noggin the Nog,0.813328
Rock'n Rodeo: Der Festivalmanager,0.804572


In [18]:
games.loc[games['Name']=='Dinosaur Island']

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumAwards,NumImplementations,NumFans,NumPageViews,RulesPosts,TotalPosts,IsExpansion,IsReimplementation,Family,Kickstarted,ImagePath,Rank:boardgame,Rank:thematic,Rank:strategygames,Rank:wargames,Rank:familygames,Rank:cgs,Rank:abstracts,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens,Playtime
144,221194,Dinosaur Island,dinosaur island player collect dna research dn...,2017,3.0319,7.69826,7.38732,1.33423,1,4,10.44186,3.357143,3,"[2, 3, 4]",21363,765,4734,376,120,90,120,8,12673,2334,5,7,8,2,872,1156109,238,783,0,0,Dinosaur Island,1,https://cf.geekdo-images.com/hLJv1A1TwrMrLLIJh...,145,42,113,21369,21369,21369,21369,21369,21369,1,1,0,0,0,0,0,0,210.0


In [75]:
# some observations

temp = master_games.loc[[4815,209685]]
sum_overs = pd.DataFrame(temp.sum()>0)
sums = list(sum_overs.loc[sum_overs[0]==True].index)
temp[sums]

,GameWeight,AvgRating,BayesAvgRating,BestPlayers,Playtime,Cat:War,"Deck, Bag, and Pool Building",Hand Management,Hexagon Grid,Set Collection,Drafting,Dice Rolling,Simulation,Contracts,Century (Plan B Games),Emerson Matsuuchi,Richard H. Berg,Card Game
BGGId,,,,,,,,,,,,,,,,,,
4815,1.09955,0.057326,-0.063502,0.0,1.5000,0.75,0.000000,0.000000,0.719857,0.000000,0.000000,0.499252,0.75,0.00,0.0,0.0,0.3,0.00
209685,-0.36020,0.142056,0.136086,0.1,0.3125,0.00,0.535117,0.318518,0.000000,0.368845,0.418282,0.000000,0.00,0.75,0.3,0.3,0.0,0.75


In [76]:
cosine_similarity(temp)

array([[1.        , 0.14465309],
       [0.14465309, 1.        ]])

In [77]:
# checking stuff

games.loc[games['BGGId']==1313]

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumAwards,NumImplementations,NumFans,NumPageViews,RulesPosts,TotalPosts,IsExpansion,IsReimplementation,Family,Kickstarted,ImagePath,Rank:boardgame,Rank:thematic,Rank:strategygames,Rank:wargames,Rank:familygames,Rank:cgs,Rank:abstracts,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens,Playtime
5656,1313,1826: Railroading in France and Belgium from 1826,xx game set france belgium reflect period fr...,2000,3.8571,7.41667,5.65715,1.24199,2,6,14.0,NaN,0,[],207,81,103,14,210,210,210,14,144,68,0,0,0,0,5,38199,6,12,0,0,18xx,0,https://cf.geekdo-images.com/Ui81G3K_velgFBuGI...,5700,21369,1737,21369,21369,21369,21369,21369,21369,0,1,0,0,0,0,0,0,360.0


# FUTURE WORK

- Incorporate the description for better matching
- Incorporate the user comments for better matching